# 04 - Modelagem Preditiva
Autora: Fernanda Baptista de Siqueira  
Curso: MBA em Tecnologia para Negócios – AI, Data Science e Big Data  
Tema: Análise de Acidentes de Trânsito em Porto Alegre (2020–2024)  
Origem DataFrame: Equipe Armazém de Dados de Mobilidade - EAMOB/CIET  
https://dadosabertos.poa.br/dataset/acidentes-de-transito-acidentes (11/05/2025)  

### 1. Importa bibliotecas e funções. Carrega dados

In [1]:
from config import (
    pd, sns, plt, np,
    resumo_df, ajustar_tipos, 
    PATH_CLEAN, COLS_VEICULOS
)

from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, PoissonRegressor
from imblearn.over_sampling import SMOTE
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from xgboost import XGBClassifier

df = pd.read_parquet(PATH_CLEAN + "df_limpo_chuva.parquet")
resumo_df(df)


Dimensões: (68837, 35)

Tipos de dados:
predial1                   Int32
queda_arr                  Int32
data              datetime64[ns]
feridos                    Int32
feridos_gr                 Int32
fatais                     Int32
auto                       Int32
taxi                       Int32
lotacao                    Int32
onibus_urb                 Int32
onibus_met                 Int32
onibus_int                 Int32
caminhao                   Int32
moto                       Int32
carroca                    Int32
bicicleta                  Int32
outro                      Int32
cont_vit                   Int32
ups                        Int32
patinete                   Int32
idacidente                 Int32
log1              string[python]
log2              string[python]
tipo_acid               category
dia_sem                 category
hora             timedelta64[ns]
noite_dia               category
regiao                  category
hora_int                   int64
dat

,predial1,queda_arr,data,feridos,feridos_gr,fatais,auto,taxi,lotacao,onibus_urb,onibus_met,onibus_int,caminhao,moto,carroca,bicicleta,outro,cont_vit,ups,patinete,idacidente,log1,log2,tipo_acid,dia_sem,hora,noite_dia,regiao,hora_int,data_hora,total_vitimas,soma_veiculos,data_meteo,chuva,chovendo
0,2500,0,2020-01-01,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,669196,AV FARRAPOS,AV SAO PEDRO,ABALROAMENTO,Quarta,0 days 02:20:00,NOITE,NORTE,2,2020-01-01 02:20:00,0,2,2020-01-01 02:00:00,0.00,0
1,598,0,2020-01-01,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,5,0,669089,AV BENTO GONCALVES,<NA>,ABALROAMENTO,Quarta,0 days 03:00:00,NOITE,LESTE,3,2020-01-01 03:00:00,1,2,2020-01-01 03:00:00,0.00,0
2,0,0,2020-01-01,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,669206,R SANTA FLORA,AV DA CAVALHADA,COLISÃO,Quarta,0 days 17:15:00,DIA,SUL,17,2020-01-01 17:15:00,0,2,2020-01-01 17:00:00,0.40,1
3,399,0,2020-01-01,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,669195,R SAO FRANCISCO DE ASSIS,<NA>,EVENTUAL,Quarta,0 days 17:15:00,DIA,NORTE,17,2020-01-01 17:15:00,0,1,2020-01-01 17:00:00,5.70,1
4,400,0,2020-01-01,1,1,0,0,0,0,0,0,0,0,1,0,1,0,1,5,0,683303,AV SENADOR TARSO DUTRA,<NA>,ABALROAMENTO,Quarta,0 days 23:00:00,NOITE,LESTE,23,2020-01-01 23:00:00,1,2,2020-01-01 23:00:00,0.00,0


### 2. Valida colunas

In [12]:
# Garantir formato de data
df["data"] = pd.to_datetime(df["data"], errors="coerce")

# Derivar colunas úteis
df["ano"] = df["data"].dt.year
df["mes"] = df["data"].dt.month
df["ano_mes"] = df["data"].dt.to_period("M").astype(str)

# Alvo deve ser não-negativo
assert (df["ups"] >= 0).all(), "ups contém valores negativos; verifique o pré-processamento."


### 3) Seleciona features

In [13]:
# aplicar tipagem padrão
df = ajustar_tipos(df)

# seleção de features
cols_temporais = ["dia_sem", "hora_int", "noite_dia"]
cols_geo      = ["regiao", "log1"]
cols_clima    = ["chuva", "chovendo"]

features = cols_temporais + cols_geo + cols_clima + COLS_VEICULOS
target   = "ups"

X = df[features].copy()
y = df[target].astype(float).copy()

print("Features selecionadas:", features)


Features selecionadas: ['dia_sem', 'hora_int', 'noite_dia', 'regiao', 'log1', 'chuva', 'chovendo', 'auto', 'taxi', 'lotacao', 'onibus_urb', 'onibus_met', 'onibus_int', 'caminhao', 'moto', 'carroca', 'bicicleta', 'outro', 'patinete']


4) Split temporal: treino/validação/teste
* Treino: 2020–2023  
* Validação: 2024   
* Teste: 2025 (jan–mai)

In [14]:
# Filtrar 2025 até maio (se houver 2025)
df_treino = df[(df["ano"] >= 2020) & (df["ano"] <= 2023)]
df_val    = df[df["ano"] == 2024]
df_teste  = df[df["ano"] == 2025]
if not df_teste.empty:
    df_teste = df_teste[df_teste["mes"] <= 5]

def xy(dfx):
    X = dfx[features].copy()
    y = dfx[target].astype(float).copy()
    return X, y

X_train, y_train = xy(df_treino)
X_val,   y_val   = xy(df_val)
X_test,  y_test  = xy(df_teste)

print("Shapes:",
      "train", X_train.shape,
      "val",   X_val.shape,
      "test",  X_test.shape)


Shapes: train (50718, 19) val (14836, 19) test (3283, 19)


5) Pré-processamento (numérico + categórico) e modelos

In [17]:
# Inferir tipos
cat_cols = X_train.select_dtypes(include=["category","object"]).columns.tolist()
num_cols = [c for c in X_train.columns if c not in cat_cols]

# Pipelines
numeric = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler",  StandardScaler(with_mean=False))  # with_mean False para segurança em esparsidade
])

categorical = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe",     OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric, num_cols),
        ("cat", categorical, cat_cols)
    ]
)

modelos = {
    "LinearRegression": LinearRegression(),
    "PoissonRegressor": PoissonRegressor(alpha=1.0, max_iter=1000),  # requer y>=0 (ok p/ UPS)
    "RandomForest": RandomForestRegressor(n_estimators=300, max_depth=None, random_state=42, n_jobs=-1),
    "HistGBR": HistGradientBoostingRegressor(random_state=42)
}


6) Treino, avaliação e seleção

In [18]:
def avaliar(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    return {"MAE": mae, "RMSE": rmse, "R2": r2}

resultados = {}

for nome, base_model in modelos.items():
    pipe = Pipeline(steps=[("prep", preprocessor), ("model", base_model)])
    pipe.fit(X_train, y_train)

    yhat_val = pipe.predict(X_val)
    met = avaliar(y_val, yhat_val)
    resultados[nome] = {"pipeline": pipe, "val": met}

# Ranking
ranking = (pd.DataFrame({k: v["val"] for k,v in resultados.items()})
             .T.sort_values(by=["RMSE","MAE","R2"], ascending=[True, True, False]))
ranking


ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'AV FARRAPOS'

Escolher menor RMSE ou MAE e maior r² na validação.

In [ ]:
melhor_nome = ranking.index[0]
best_pipe = resultados[melhor_nome]["pipeline"]
print("🏆 Melhor modelo (val):", melhor_nome, resultados[melhor_nome]["val"])

# Avaliar no teste (se existir)
if not X_test.empty:
    yhat_test = best_pipe.predict(X_test)
    print("Teste:", avaliar(y_test, yhat_test))


7) Importância de features & interpretabilidade

Para árvores, usamos feature_importances_. Para modelos lineares, coeficientes.
Se você tiver SHAP instalado, incluí um bloco opcional.

In [ ]:
def nomes_features_transformados(preprocessor):
    # Recupera nomes após OneHotEncoder
    out = []
    # num
    out += num_cols
    # cat
    ohe = preprocessor.named_transformers_["cat"].named_steps["ohe"]
    cat_feat_names = ohe.get_feature_names_out(cat_cols).tolist()
    out += cat_feat_names
    return out

# Importância (quando aplicável)
try:
    model = best_pipe.named_steps["model"]
    feat_names = nomes_features_transformados(best_pipe.named_steps["prep"])

    if hasattr(model, "feature_importances_"):
        imp = pd.Series(model.feature_importances_, index=feat_names).sort_values(ascending=False).head(20)
        ax = imp[::-1].plot(kind="barh")
        ax.set_title(f"Top 20 importâncias — {melhor_nome}")
        plt.show()

    elif hasattr(model, "coef_"):
        coefs = pd.Series(model.coef_.ravel(), index=feat_names).sort_values()
        ax = coefs.head(10).plot(kind="barh"); plt.title(f"Coef. (neg) — {melhor_nome}"); plt.show()
        ax = coefs.tail(10).plot(kind="barh"); plt.title(f"Coef. (pos) — {melhor_nome}"); plt.show()
except Exception as e:
    print("Importância/coeficientes não disponíveis:", e)


SHAP (SHapley Additive exPlanations) é uma ferramenta poderosa para interpretar modelos de machine learning, especialmente modelos complexos como árvores de decisão e redes neurais. Ele atribui a cada feature uma contribuição para a previsão do modelo, permitindo entender como cada variável influencia o resultado.

In [ ]:
if HAS_SHAP and hasattr(best_pipe.named_steps["model"], "predict"):
    # Amostra para reduzir custo
    amostra = X_val.sample(min(3000, len(X_val)), random_state=42)
    X_val_trans = best_pipe.named_steps["prep"].transform(amostra)

    # Escolha do explainer depende do modelo
    try:
        explainer = shap.Explainer(best_pipe.named_steps["model"])
        shap_values = explainer(X_val_trans)
        shap.plots.beeswarm(shap_values, max_display=20)
    except Exception as e:
        print("SHAP não pôde rodar com este modelo:", e)
else:
    print("SHAP indisponível (biblioteca ausente) — pulando.")


8) Previsão temporal (mensal) com SARIMAX (opcional)

Previsão do total mensal de UPS para avaliar tendência de severidade.
Ajuste simples; melhore com covariáveis exógenas (chuva média mensal).

In [ ]:
if HAS_STATSMODELS:
    # Série mensal de UPS
    s_mensal = (df
                .dropna(subset=["data"])
                .set_index("data")
                .resample("M")["ups"].sum())

    # Treino até 2023, valida 2024, teste 2025-05
    s_train = s_mensal.loc[: "2023-12-31"]
    s_val   = s_mensal.loc["2024-01-01":"2024-12-31"]
    s_test  = s_mensal.loc["2025-01-01":"2025-05-31"]

    # Exógenas (chuva média mensal), se existir
    if "chuva" in df.columns:
        exo = df.set_index("data").resample("M")["chuva"].mean()
        exo_train = exo.loc[s_train.index]
        exo_val   = exo.loc[s_val.index]
        exo_test  = exo.loc[s_test.index] if not s_test.empty else None
    else:
        exo_train = exo_val = exo_test = None

    # Modelo SARIMAX simples
    mod = sm.tsa.statespace.SARIMAX(
        s_train, order=(1,1,1), seasonal_order=(1,1,1,12),
        exog=exo_train, enforce_stationarity=False, enforce_invertibility=False
    )
    res = mod.fit(disp=False)

    pred_val = res.get_forecast(steps=len(s_val), exog=exo_val)
    pred_mean_val = pred_val.predicted_mean
    ci_val = pred_val.conf_int()

    ax = s_mensal.plot(label="observado", alpha=0.6)
    pred_mean_val.plot(ax=ax, label="previsto (val)")
    ax.fill_between(ci_val.index, ci_val.iloc[:,0], ci_val.iloc[:,1], alpha=0.2)
    ax.axvspan(pd.Timestamp("2024-01-01"), pd.Timestamp("2024-12-31"), color="orange", alpha=0.1, label="val")
    if not s_test.empty:
        ax.axvspan(pd.Timestamp("2025-01-01"), pd.Timestamp("2025-05-31"), color="green", alpha=0.1, label="teste")
    ax.set_title("UPS mensal — observado vs previsão (SARIMAX)")
    ax.legend(); plt.show()

    # Erros em validação
    from math import sqrt
    rmse_val = sqrt(((s_val - pred_mean_val)**2).mean())
    mae_val = (s_val - pred_mean_val).abs().mean()
    print({"RMSE_val_mensal": rmse_val, "MAE_val_mensal": mae_val})
else:
    print("statsmodels indisponível — pulando SARIMAX.")


9) Conclusões e próximos passos (guia)

Desempenho: compare MAE/RMSE/R² entre os modelos; escolha o melhor (geralmente HistGBR e RF vão bem).

Interpretabilidade: use importâncias e (se possível) SHAP para entender sinais/efeitos.

Aprimoramentos:

Features: harmônicos de hora (seno/cosseno), sazonalidade (mês), feriados, interação chuva×noite_dia.

Espaço: agrupar log1 para reduzir cardinalidade (top-k + “outros”).

Validação: CV em blocos temporais (TimeSeriesSplit) além do hold-out por ano.

Incerteza: intervalos por quantile regression (HistGBR loss="quantile") para cenários pessimista/otimista.

Temporais: enriquecer SARIMAX com exógenas (chuva, feriados, mobilidade) e comparar com Prophet ou AutoARIMA (pmdarima).

---------

## 2. Definição do Target e Features

- Target: `ups` (Índice de Severidade Ponderado — DENATRAN, 1987).
- Features: condições climáticas, temporais, geográficas e veículos.


In [ ]:
target = "ups"

# Exemplo de seleção inicial de features
cols_temporais = ["ano", "mes", "dia_semana", "turno"]
cols_geo = ["regiao", "log1"]
cols_clima = ["precipitacao", "chuva_dia"]
from config import cols_veiculos  # já definido no config

features = cols_temporais + cols_geo + cols_clima + cols_veiculos
X = df[features]
y = df[target]


### 3. Separa treino, validação e teste

## 3. Divisão Treino / Validação / Teste

- Treino: 2020–2023  
- Validação: 2024  
- Teste: jan–mai/2025


In [ ]:
df["ano"] = df["ano"].astype(int)

df_train = df[df["ano"] <= 2023]
df_val = df[df["ano"] == 2024]
df_test = df[df["ano"] == 2025]

X_train, y_train = df_train[features], df_train[target]
X_val, y_val = df_val[features], df_val[target]
X_test, y_test = df_test[features], df_test[target]


### 4. Pré-processamento

## 4. Pré-Processamento

- Codificação de variáveis categóricas
- Escalonamento de variáveis numéricas
- Balanceamento do target com SMOTE


In [6]:
cat_features = X_train.select_dtypes(include=["object", "category"]).columns.tolist()
num_features = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
    ]
)

smote = SMOTE(random_state=42)


### 5. Modelos Candidatos

In [7]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, class_weight="balanced"),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
}


### 6. Avaliação dos Modelos

In [ ]:
results = {}

for name, model in models.items():
    pipe = Pipeline(steps=[("preprocessor", preprocessor),
                           ("model", model)])
    X_res, y_res = smote.fit_resample(X_train, y_train)
    pipe.fit(X_res, y_res)
    y_pred = pipe.predict(X_val)
    y_proba = pipe.predict_proba(X_val)[:, 1] if hasattr(pipe, "predict_proba") else None
    
    results[name] = {
        "classification_report": classification_report(y_val, y_pred, output_dict=True),
        "roc_auc": roc_auc_score(y_val, y_proba) if y_proba is not None else None
    }

results


### 7. Interpretação e Importância das Features

In [ ]:
import shap

# Exemplo com RandomForest
best_model = RandomForestClassifier(n_estimators=200, random_state=42)
pipe = Pipeline([("preprocessor", preprocessor), ("model", best_model)])
X_res, y_res = smote.fit_resample(X_train, y_train)
pipe.fit(X_res, y_res)

explainer = shap.TreeExplainer(best_model)
X_val_trans = preprocessor.transform(X_val)
shap_values = explainer.shap_values(X_val_trans)
shap.summary_plot(shap_values, X_val_trans, feature_names=preprocessor.get_feature_names_out())


### 8. Modelagem Temporal

In [ ]:
import statsmodels.api as sm

# Série mensal
df_mensal = df.groupby(["ano_mes"])["ups"].sum()
train = df_mensal[:'2023-12']
val = df_mensal['2024']
test = df_mensal['2025-05']

# SARIMAX (exemplo simples)
model = sm.tsa.statespace.SARIMAX(train, order=(1,1,1), seasonal_order=(1,1,1,12))
sarimax_res = model.fit(disp=False)
forecast = sarimax_res.get_forecast(steps=len(val))
forecast_ci = forecast.conf_int()


### 9. Discussão e Limitações

- Comparar interpretabilidade (Regressão) vs performance (Ensembles).
- Destacar importância de incerteza (Bao et al., 2020).
- Ressaltar que modelos preditivos não garantem causalidade (Pearl et al., 2016).
- Indicar próximos passos: ensembles híbridos, variáveis externas de mobilidade urbana.


---------------------------

03. Pré processamento
* seleção de features
* balanceamento (SMOTE, undersampling)
* normalização

4. Modelos candidatos
* Classificação: Regressão Logística, Random Forest, XGBoost
* Series Temporais: SARIMA, Prophet

5. Treinamento e validação
* Separação treino/teste
* Treinar modelos, guardar métricas (accuracy, recall, AUC)
* Comparar modelos

6. Interpretação
* Importância das variáveis (ex.: feature_importances_ da Random Forest)
* Gráficos de desempenho (ROC, matriz de confusão)

7. Previsão para 2025
* Usar o modelo escolhido para prever
* Comparar com dados reais de 2025 quando disponíveis.

## Referenciais Teóricos

- Breiman (2001): *Two Cultures* → interpretação vs predição.
- Bishop (2006), Hastie, Tibshirani & Friedman (2009), Murphy (2012): fundamentos estatísticos e probabilísticos.
- Géron (2023), Müller & Guido (2016), Faceli et al. (2021): boas práticas em pipelines e scikit-learn.
- Zabala (2019, 2021): aplicações de modelagem preditiva.
- Pearl et al. (2016): inferência causal.
- Vilone & Longo (2020): interpretabilidade.
- Bao et al. (2020): incerteza espaço-temporal.
- Chen et al. (2025): ensembles avançados.